In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable,grad
import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
x_max = 35.
y_max = 10.
z_max = 5.
t_max = 5.
clad_h = 1.

#laser 
x0 = 5.0
y0 = 5.0
v = 5.
r = 1.
P = 300.
eta = 0.43

rho = 8.44e-3
Cp = 0.41
k = 0.0098
h = 1e-5

In [4]:
def input_transform(X,X_max,X_min):
    X = 2.*(X-X_min)/(X_max-X_min) - 1.
    return X

def output_transform(X,hard_BC=None,inputs=None,X_max=None,X_min=None):
    X = 1000*X
    if hard_BC:
        X = X*hard_BC(inputs,X_max,X_min)
    return X

def BC_substrate(inputs,X_max,X_min):
    [x_max,y_max,z_max,t_max] = X_max
    [x_min,y_min,z_min,t_min] = X_min
    x_c = (2/(1+torch.exp(-2*1e2/x_max*(inputs[:,0:1]-x_min)))-1)*(2/(1+torch.exp(-2*1e2/x_max*(x_max-inputs[:,0:1])))-1)
    y_c = (2/(1+torch.exp(-2*1e2/y_max*(inputs[:,1:2]-y_min)))-1)*(2/(1+torch.exp(-2*1e2/y_max*(y_max-inputs[:,1:2])))-1)
    z_c = (2/(1+torch.exp(-2*1e2/z_max*(inputs[:,2:3]-t_min)))-1)
    t_c = (2/(1+torch.exp(-2*1e2/t_max*(inputs[:,3:4]-z_min)))-1)
    return x_c*y_c*z_c*t_c

In [ ]:
class FNN(nn.Module):
    def __init__(self,layers,activation,X_max=None,X_min=None,in_tf=None,out_tf=None,BC=None):
        super().__init__()
        self.activation = activation
        self.linears = nn.ModuleList()
        for i in range(1,len(layers)):
            self.linears.append(nn.Linear(layers[i-1],layers[i]))
            nn.init.xavier_uniform_(self.linears[-1].weight)
            nn.init.zeros_(self.linears[-1].bias)
        self.last = nn.Softplus()
        
        self.X_max = X_max
        self.X_min = X_min
        self.in_tf = in_tf
        self.out_tf = out_tf
        self.BC = BC
                      
    def forward(self,inputs):
        X = inputs
        # input transformation
        if self.in_tf:
            X = self.in_tf(X,self.X_max,self.X_min)
        # linear layers    
        for linear in self.linears[:-1]:
            X = self.activation(linear(X))
        # last layer
        X = self.last(self.linears[-1](X))
        # output transformation
        if self.out_tf:
            X = self.out_tf(X,self.BC,inputs,self.X_max,self.X_min)
        return X

In [ ]:
def PDE(x,y,z,t,net):
    X = torch.concat([x,y,z,t],axis=-1)
    T = net(X)
    
    T_t = grad(T,t,create_graph=True,grad_outputs=torch.ones_like(T))[0]

    T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_xx = grad(T_x,x,create_graph=True,grad_outputs=torch.ones_like(T_x))[0]
    
    T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_yy = grad(T_y,y,create_graph=True,grad_outputs=torch.ones_like(T_y))[0]
    
    T_z = grad(T,z,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_zz = grad(T_z,z,create_graph=True,grad_outputs=torch.ones_like(T_z))[0]
    
    
    Q = P*eta/(torch.pi*r**2*clad_h)*((x-x0-v*t)**2+(y-y0)**2<r**2)*(z>z_max)
    f = rho*Cp*T_t - k*(T_xx+T_yy+T_zz) - Q

    return f,Q

In [ ]:
substrate_net = FNN([4,64,64,64,1],nn.Tanh(),
                      X_max=torch.tensor([35.,10.,5.,5.]),X_min=torch.tensor([0.,0.,0.,0.]),
                      in_tf=input_transform,out_tf=output_transform,BC=BC_substrate)
clad_net = FNN([4,64,64,64,1],nn.Tanh(),
                      X_max=torch.tensor([30.,6.,6.,5.]),X_min=torch.tensor([5.,4.,5.,0.]),
                      in_tf=input_transform,out_tf=output_transform)

In [ ]:
def uniform_boundary(res,X_min,X_max,loc,t):
    if loc == '-x':
        yp = np.linspace(X_min[1],X_max[1],round((X_max[1]-X_min[1])/res) + 1)
        zp = np.linspace(X_min[2],X_max[2],round((X_max[2]-X_min[2])/res) + 1)
        grid = np.meshgrid(yp,zp)
        yp = grid[0].flatten()
        zp = grid[1].flatten()
        xp = np.ones_like(yp)*X_min[0]
        x = np.vstack((xp,yp,zp)).T
    if loc == '+x':
        yp = np.linspace(X_min[1],X_max[1],round((X_max[1]-X_min[1])/res) + 1) 
        zp = np.linspace(X_min[2],X_max[2],round((X_max[2]-X_min[2])/res) + 1)
        grid = np.meshgrid(yp,zp)
        yp = grid[0].flatten()
        zp = grid[1].flatten()
        xp = np.ones_like(yp)*X_max[0]
        x = np.vstack((xp,yp,zp)).T
    if loc == '-y':
        xp = np.linspace(X_min[0],X_max[0],round((X_max[0]-X_min[0])/res) + 1)
        zp = np.linspace(X_min[2],X_max[2],round((X_max[2]-X_min[2])/res) + 1)
        grid = np.meshgrid(xp,zp)
        xp = grid[0].flatten()
        zp = grid[1].flatten()
        yp = np.ones_like(xp)*X_min[1]
        x = np.vstack((xp,yp,zp)).T
    if loc == '+y':
        xp = np.linspace(X_min[0],X_max[0],round((X_max[0]-X_min[0])/res) + 1)
        zp = np.linspace(X_min[2],X_max[2],round((X_max[2]-X_min[2])/res) + 1)
        grid = np.meshgrid(xp,zp)
        xp = grid[0].flatten()
        zp = grid[1].flatten()
        yp = np.ones_like(xp)*X_max[1]
        x = np.vstack((xp,yp,zp)).T
    if loc == '-z':
        xp = np.linspace(X_min[0],X_max[0],round((X_max[0]-X_min[0])/res) + 1)
        yp = np.linspace(X_min[1],X_max[1],round((X_max[1]-X_min[1])/res) + 1)
        grid = np.meshgrid(xp,yp)
        xp = grid[0].flatten()
        yp = grid[1].flatten()
        zp = np.ones_like(xp)*X_min[2]
        x = np.vstack((xp,yp,zp)).T
    if loc == '+z':
        xp = np.linspace(X_min[0],X_max[0],round((X_max[0]-X_min[0])/res) + 1) 
        yp = np.linspace(X_min[1],X_max[1],round((X_max[1]-X_min[1])/res) + 1) 
        grid = np.meshgrid(xp,yp)
        xp = grid[0].flatten()
        yp = grid[1].flatten()
        zp = np.ones_like(xp)*X_max[2]
        x = np.vstack((xp,yp,zp)).T
    if loc == 'domain':
        xp = np.linspace(X_min[0],X_max[0],round((X_max[0]-X_min[0])/res) + 1) 
        yp = np.linspace(X_min[1],X_max[1],round((X_max[1]-X_min[1])/res) + 1)
        zp = np.linspace(X_min[2],X_max[2],round((X_max[2]-X_min[2])/res) + 1)
        grid = np.meshgrid(xp,yp,zp)
        xp = grid[0].flatten()
        yp = grid[1].flatten()
        zp = grid[2].flatten()
        x = np.vstack((xp,yp,zp)).T
        
    xt = []
    num = x.shape[0]
    for ti in t:
        xt.append(np.hstack((x, np.full([num,1], ti))))
    xt = np.vstack(xt) 
    return xt, xt.shape[0]

In [ ]:
t = np.linspace(0,t_max,101)
bound_1,_ = uniform_boundary(1,[0,0,0],[5,10,5],'+z',t)
bound_2,_ = uniform_boundary(1,[30,0,0],[35,10,5],'+z',t)
bound_3,_ = uniform_boundary(1,[5,0,0],[30,4,5],'+z',t)
bound_4,_ = uniform_boundary(1,[5,6,0],[30,10,5],'+z',t)
bound_5 = []
for ti in t:
    try:
        xi,_ = uniform_boundary(0.5,[x0+ti*v+r,4,z_max],[30,6,z_max],'+z',[ti])
        bound_5.append(xi)
    except:
        continue
        
bound_5 = np.vstack(bound_5)
bound_sub = np.vstack([bound_1,bound_2,bound_3,bound_4,bound_5])

In [ ]:
t = np.linspace(0,t_max,101)
bound_1,_ = uniform_boundary(0.1,[5,4,5],[5,6,6],'-x',t)
bound_2 = []
for ti in t[1:]:
    temp1,_ = uniform_boundary(0.1,[min(x0+ti*v+r,30),4,5],[min(x0+ti*v+r,30),6,6],'+x',[ti])
    temp2,_ = uniform_boundary(0.1,[5,4,5],[min(x0+ti*v+r,30),4,6],'-y',[ti])
    temp3,_ = uniform_boundary(0.1,[5,6,5],[min(x0+ti*v+r,30),6,6],'+y',[ti])
    temp2,_ = uniform_boundary(0.1,[5,4,5],[min(x0+ti*v+r,30),4,6],'-y',[ti])
    temp3,_ = uniform_boundary(0.1,[5,6,5],[min(x0+ti*v+r,30),6,6],'+y',[ti])
bound_1.shape